In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
import pandas as pd
import numpy as np
from bayesian.train_bn import structure_learning, parameter_learning, parameter_learning_mix
from preprocess.discretization import get_nodes_type, discretization, inverse_discretization, code_categories
from bayesian.save_bn import save_structure, save_params, read_structure, read_params
from external.libpgm.hybayesiannetwork import HyBayesianNetwork
from visualization.visualization import draw_BN
from bayesian.calculate_accuracy import calculate_acc
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from copy import copy
from external.libpgm.sampleaggregator import SampleAggregator
import operator
from sklearn.metrics import accuracy_score, mean_squared_error
from scipy import stats
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
import math
from pomegranate import DiscreteDistribution
import random
from sklearn.model_selection import train_test_split
import time

In [3]:
vk = pd.read_csv('vk_profiles_cross.csv')
vk.head(10)

,id,sex,age,has_high_education,relation,num_of_relatives,followers_count,status,mobile_phone,twitter,...,interests,interests_topic,movies,music,quotes,personal_alcohol,personal_life_main,personal_people_main,personal_political,age_hidden
0,75,-1,32,-1,0,2,4068,11,-1,-1,...,0,0,-1,-1,-1,0,0,0,0,-1
1,172,1,35,-1,0,2,0,0,-1,-1,...,0,0,-1,-1,-1,0,0,0,0,-1
2,175,1,0,-1,0,2,87224,0,-1,-1,...,0,0,-1,-1,-1,0,0,0,0,1
3,176,1,37,-1,0,2,0,0,-1,-1,...,0,0,-1,-1,-1,0,0,0,0,-1
4,252,1,39,1,4,2,270,11,-1,-1,...,0,0,-1,-1,-1,0,0,0,5,-1
5,375,1,44,-1,0,2,476,0,-1,-1,...,0,0,-1,-1,-1,0,0,0,0,-1
6,505,1,31,-1,0,2,0,0,-1,-1,...,0,0,-1,-1,-1,0,0,0,0,-1
7,558,-1,38,-1,0,2,0,0,-1,-1,...,0,0,-1,-1,-1,0,0,0,0,-1
8,609,1,34,-1,0,2,184,11,-1,-1,...,0,0,-1,-1,-1,0,0,0,0,-1
9,711,1,35,-1,0,2,0,0,-1,-1,...,0,0,-1,-1,-1,0,0,0,0,-1


In [4]:
interes = pd.read_csv('interests_cross.csv')
interes.head(10)

,sbj_0_0,sbj_0_1,sbj_0_2,sbj_0_3,sbj_0_4,sbj_0_5,sbj_0_6,sbj_0_7,sbj_0_8,sbj_0_11,...,sbj_0_16,sbj_0_21,sbj_0_22,sbj_0_24,sbj_0_25,sbj_0_26,sbj_0_28,sbj_0_29,sbj_0_30,user_id
0,0.080425,0.055359,0.099797,0.022773,0.015482,0.031199,0.043981,0.030139,0.001985,0.028075,...,0.016505,0.097016,0.052347,0.083770,0.080065,0.043225,0.007930,0.050312,0.033875,75
1,0.029986,0.094844,0.355780,0.023383,0.002954,0.022161,0.062144,0.060318,0.009339,0.015776,...,0.018903,0.009919,0.011701,0.013376,0.068087,0.023920,0.035157,0.028263,0.012047,172
2,0.035605,0.061990,0.095158,0.038652,0.003878,0.021765,0.049660,0.086779,0.048905,0.031807,...,0.034216,0.015246,0.020291,0.014943,0.045807,0.040546,0.029588,0.028808,0.013248,175
3,0.077533,0.055265,0.019700,0.004943,0.022406,0.004232,0.014765,0.053713,0.002173,0.022065,...,0.033823,0.171002,0.078734,0.100349,0.071107,0.024383,0.008047,0.008630,0.071860,176
4,0.023390,0.120930,0.329887,0.044455,0.000952,0.094942,0.012271,0.024358,0.001795,0.018554,...,0.001795,0.027650,0.023757,0.001456,0.066031,0.056551,0.001863,0.040104,0.001229,252
5,0.034694,0.067079,0.130919,0.100531,0.001699,0.017660,0.221572,0.062680,0.006153,0.012825,...,0.006846,0.005404,0.012680,0.005205,0.035175,0.086706,0.009330,0.066363,0.005775,375
6,0.023996,0.095135,0.227457,0.007781,0.003309,0.012939,0.040225,0.071300,0.019517,0.013690,...,0.071261,0.005601,0.018104,0.003592,0.065593,0.005357,0.118209,0.022737,0.016910,505
7,0.028281,0.160249,0.166804,0.015272,0.009584,0.037353,0.025442,0.030182,0.007193,0.077362,...,0.012782,0.047231,0.031132,0.064004,0.041184,0.025353,0.039849,0.033195,0.008809,558
8,0.031313,0.148632,0.118563,0.018726,0.003604,0.021363,0.048178,0.053077,0.003177,0.023939,...,0.030237,0.023843,0.022334,0.018750,0.040905,0.122373,0.043709,0.091793,0.008117,609
9,0.019280,0.121727,0.159137,0.012697,0.009487,0.024468,0.015715,0.056650,0.009854,0.016708,...,0.013159,0.019522,0.012918,0.013523,0.027085,0.040660,0.308797,0.031134,0.004571,711


In [5]:
interes = interes.drop(columns = ['user_id'])

In [6]:
finance = pd.read_csv('transactions_cross.csv')
finance.head(10)

,CustomerKey,max_tr,min_tr,mean_tr,median_tr,90_perc,sum_am,tr_per_month,cash_sum,cash_usage,game_sum,is_gamer,parent_sum,is_parent,driver_sum,is_driver,pets_sum,has_pets
0,75,21290.92,45.00,1778.056571,900.0,2994.440,62231.98,2.759820,3400.00,1,0.00,-1,0.00,-1,0.0,-1,0.0,-1
1,172,74377.68,5.00,991.952426,400.0,1652.980,2007711.71,96.407254,379300.00,1,0.00,-1,0.00,-1,0.0,-1,19383.0,-1
2,175,124500.00,291.85,18155.974390,6117.0,40000.000,744394.95,2.381511,18800.00,1,0.00,-1,28466.60,1,0.0,-1,0.0,-1
3,176,15600.00,15.00,1310.183217,249.5,4750.000,411397.53,15.050675,266600.00,1,0.00,-1,0.00,-1,0.0,-1,230.0,-1
4,252,139845.00,68.20,9718.394202,1390.0,27760.000,1156488.91,5.986757,31184.40,1,7420.15,1,0.00,-1,0.0,-1,0.0,-1
5,375,2400.00,35.00,1173.363636,1000.0,2300.000,12907.00,0.534833,8470.00,1,0.00,-1,0.00,-1,0.0,-1,0.0,-1
6,505,119084.00,1.00,3878.404798,1424.5,6015.000,2982493.29,37.509546,74000.00,1,11370.75,1,1066.86,1,0.0,-1,21663.0,-1
7,558,7870.00,64.00,1459.894364,1000.0,3378.800,80294.19,2.704407,0.00,-1,0.00,-1,59992.53,1,0.0,-1,0.0,-1
8,609,129550.00,19.90,1646.404308,426.5,2414.568,642097.68,19.459641,125400.00,1,0.00,-1,0.00,-1,0.0,-1,0.0,-1
9,711,52500.00,0.63,1339.101724,500.0,2499.800,1475690.10,52.572784,453432.86,1,0.00,-1,0.00,-1,0.0,-1,516.0,-1


In [7]:
finance = finance.drop(columns=['CustomerKey'])

In [8]:
all_data = pd.concat([vk, interes, finance], axis=1)

In [9]:
all_data = all_data.drop(columns=['id'])

In [10]:
all_data.dropna(inplace=True)

In [11]:
all_data_sample = all_data.sample(50)

In [12]:
nodes_type = get_nodes_type(all_data)
nodes_type

{'sex': 'disc',
 'age': 'disc',
 'has_high_education': 'disc',
 'relation': 'disc',
 'num_of_relatives': 'disc',
 'followers_count': 'disc',
 'status': 'disc',
 'mobile_phone': 'disc',
 'twitter': 'disc',
 'facebook': 'disc',
 'instagram': 'disc',
 'about': 'disc',
 'about_topic': 'disc',
 'activities': 'disc',
 'activities_topic': 'disc',
 'books': 'disc',
 'interests': 'disc',
 'interests_topic': 'disc',
 'movies': 'disc',
 'music': 'disc',
 'quotes': 'disc',
 'personal_alcohol': 'disc',
 'personal_life_main': 'disc',
 'personal_people_main': 'disc',
 'personal_political': 'disc',
 'age_hidden': 'disc',
 'sbj_0_0': 'cont',
 'sbj_0_1': 'cont',
 'sbj_0_2': 'cont',
 'sbj_0_3': 'cont',
 'sbj_0_4': 'cont',
 'sbj_0_5': 'cont',
 'sbj_0_6': 'cont',
 'sbj_0_7': 'cont',
 'sbj_0_8': 'cont',
 'sbj_0_11': 'cont',
 'sbj_0_12': 'cont',
 'sbj_0_14': 'cont',
 'sbj_0_15': 'cont',
 'sbj_0_16': 'cont',
 'sbj_0_21': 'cont',
 'sbj_0_22': 'cont',
 'sbj_0_24': 'cont',
 'sbj_0_25': 'cont',
 'sbj_0_26': 'cont

In [13]:
columns_for_disc = []
for key in nodes_type.keys():
    if nodes_type[key] == 'cont':
        columns_for_disc.append(key)

In [14]:
discrete_data, coder = discretization(all_data_sample, 'equal_frequency', columns_for_disc)

C:\Users\anaxa\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 30 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
C:\Users\anaxa\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 31 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
C:\Users\anaxa\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\preprocessing\_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 32 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
C:\Users\anaxa\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\prepr

In [15]:
bn_geo = structure_learning(discrete_data, 'HC', 'K2', nodes_type)

  0%|          | 161/1000000 [01:39<171:28:58,  1.62it/s]


In [16]:
params = parameter_learning(all_data.sample(1000), nodes_type, bn_geo)

In [17]:
save_structure(bn_geo, 'geo_net')
skel = read_structure('geo_net')
save_params(params, 'geo_net_param')
params = read_params('geo_net_param')
geo_bn = HyBayesianNetwork(skel, params)

In [18]:
start = time.time()
sample = pd.DataFrame(geo_bn.randomsample(80000))
end = time.time()
print(end - start)

108.78442120552063


In [ ]:
'sex': 'disc',
 'age': 'disc',
 'has_high_education': 'disc',
 'relation': 'disc',
 'num_of_relatives': 'disc',
 'followers_count': 'disc',
 'status': 'disc',
 'mobile_phone': 'disc',
 'twitter': 'disc',
 'facebook': 'disc',
 'instagram': 'disc',
 'about': 'disc',
 'about_topic': 'disc',
 'activities': 'disc',
 'activities_topic': 'disc',
 'books': 'disc',
 'interests': 'disc',
 'interests_topic': 'disc',
 'movies': 'disc',
 'music': 'disc',
 'quotes': 'disc',
 'personal_alcohol': 'disc',
 'personal_life_main': 'disc',
 'personal_people_main': 'disc',
 'personal_political': 'disc',
 'age_hidden': 'disc',

In [21]:
sample['sex'] = sample['sex'].apply(lambda x: int(x))
sample['age'] = sample['age'].apply(lambda x: int(x))
sample['has_high_education'] = sample['has_high_education'].apply(lambda x: int(x))
sample['relation'] = sample['relation'].apply(lambda x: int(x))
sample['num_of_relatives'] = sample['num_of_relatives'].apply(lambda x: int(x))
sample['followers_count'] = sample['followers_count'].apply(lambda x: int(x))
sample['status'] = sample['status'].apply(lambda x: int(x))
sample['mobile_phone'] = sample['mobile_phone'].apply(lambda x: int(x))
sample['twitter'] = sample['twitter'].apply(lambda x: int(x))
sample['facebook'] = sample['facebook'].apply(lambda x: int(x))
sample['instagram'] = sample['instagram'].apply(lambda x: int(x))
sample['about'] = sample['about'].apply(lambda x: int(x))
sample['about_topic'] = sample['about_topic'].apply(lambda x: int(x))

In [19]:
def MMD(x, y, kernel="rbf"):
    xx, yy, zz = np.dot(x, x.T), np.dot(y, y.T), np.dot(x, y.T)

    rx = np.tile(np.expand_dims(np.diag(xx), 0), (xx.shape[0], 1))
    ry = np.tile(np.expand_dims(np.diag(yy), 0), (yy.shape[0], 1))

    dxx = rx.T + rx - 2. * xx
    dyy = ry.T + ry - 2. * yy
    dxy = rx.T + ry - 2. * zz

    XX, YY, XY = np.zeros(xx.shape), np.zeros(xx.shape), np.zeros(xx.shape)

    if kernel == "multiscale":
        bandwidth_range = [0.2, 0.5, 0.9, 1.3]
        for a in bandwidth_range:
            XX += a**2 * (a**2 + dxx)**-1
            YY += a**2 * (a**2 + dyy)**-1
            XY += a**2 * (a**2 + dxy)**-1

    if kernel == "rbf":
        bandwidth_range = [10, 15, 20, 50]
        for a in bandwidth_range:
            XX += np.exp(-0.5*dxx / a)
            YY += np.exp(-0.5*dyy / a)
            XY += np.exp(-0.5*dxy / a)

    return np.mean(XX + YY - 2. * XY)

In [20]:
MMD(all_data.sample(100).values, sample.sample(100).values)

TypeError: can't multiply sequence by non-int of type 'str'